In [13]:
import numpy as np
class River_Swim:
    def __init__(self,nS=6,nA=2):
        self.nS = nS;
        self.nA = nA;
    def gen_probability(self):
        P = np.zeros((self.nA,self.nS,self.nS));
        P[0,0,0],P[0,0,1] = 0.9,0.1
        P[1,self.nS-1,self.nS-1],P[1,self.nS-1,self.nS-2] = 0.4,0.6
        for i in range(1,self.nS-1):
            P[0,i,i-1],P[0,i,i],P[0,i,i+1] = 0.3,0.6,0.1
            P[1,self.nS-i-1,self.nS-i],P[1,self.nS-i-1,self.nS-i-1],P[1,self.nS-i-1,self.nS-i-2] = 0.3,0.6,0.1
        P[0,self.nS-1,self.nS-1],P[0,self.nS-1,self.nS-2] = 0.7,0.3
        P[1,0,0],P[1,0,1] = 0.7,0.3
        #print(P)
        #print("Reward function");
        return P;
    def gen_expected_reward(self):
        R = np.zeros((self.nA,self.nS));
        R[0,0] = 1
        R[1,self.nS-1] = 0.82
        #print(R);
        return R;

In [14]:
class beh_pol_sd:
    def __init__(self,P,policy,nS,nA):
        self.P = P
        self.policy = policy
        self.nS = nS;
        self.nA = nA;
        print(self.nS);
    
    def onehot(self):
        pol = np.zeros((self.nS,self.nA));
        for i in range(self.nS):
            pol[i][int(self.policy[i])]=1;
        return pol;
    def find_transition_matrix(self,onehot_encode=1):
        if(onehot_encode==1):
            self.policy = self.onehot()
        T_s_s_next = np.zeros((self.nS,self.nS));
        for s in range(self.nS):
            for s_next in range(self.nS):
                for a in range(self.nA):
                    #print(s,s_next,a);
                    #print(T[a,s,s_next]);
                    T_s_s_next[s,s_next]+=self.P[a,s,s_next]*self.policy[s,a];
        return T_s_s_next;
    def state_distribution_simulated(self,onehot_encode=1):
        P_policy = self.find_transition_matrix(onehot_encode)
        #print(P_policy);
        P_dash = np.append(P_policy - np.eye(self.nS),np.ones((self.nS,1)),axis=1);
        #print(P_dash);
        P_last = np.linalg.pinv(np.transpose(P_dash))[:,-1]
        return P_last;

In [15]:
class find_D_mu:
    def __init__(self,P,policy,nS,nA):
        self.P = P
        self.policy = policy
        self.nS = nS;
        self.nA = nA;
    
    def onehot(self):
        pol = np.zeros((self.nS,self.nA));
        for i in range(self.nS):
            pol[i][int(self.policy[i])]=1;
        return pol;
    def find_transition_matrix(self,onehot_encode=1):
        #print(self.policy)
        if(onehot_encode==1):
            self.policy = self.onehot()
        T_s_s_next = np.zeros((self.nS,self.nS));
        for s in range(self.nS):
            for s_next in range(self.nS):
                for a in range(self.nA):
                    #print(s,s_next,a);
                    #print(T[a,s,s_next]);
                    T_s_s_next[s,s_next]+=self.P[a,s,s_next]*self.policy[s,a];
        return T_s_s_next;
    def state_distribution_simulated(self,P_policy,policy,onehot_encode=1):
        self.policy = policy;
        P_policy = self.find_transition_matrix(onehot_encode)
        #print(P_policy);
        #print(P_policy);
        P_dash = np.append(P_policy - np.eye(self.nS),np.ones((self.nS,1)),axis=1);
        #print(P_dash);
        P_last = np.linalg.pinv(np.transpose(P_dash))[:,-1]
        return P_last;

In [16]:
nS = 6
nA = 2
batch_size = 50;
estimated_P_freq = np.zeros((nA,nS,nS));
behaviour_policy = np.ones((nS,nA))*0.5
rep_cost = 0.7
mr_obj = River_Swim(nS,nA)
P,R = mr_obj.gen_probability(),mr_obj.gen_expected_reward()
state = 0;
runs = 1;
T = 100

state = 0;
final_list=[];
temporary_list=[];
P_est = np.zeros((nA,nS,nS));

In [17]:
P

array([[[0.9, 0.1, 0. , 0. , 0. , 0. ],
        [0.3, 0.6, 0.1, 0. , 0. , 0. ],
        [0. , 0.3, 0.6, 0.1, 0. , 0. ],
        [0. , 0. , 0.3, 0.6, 0.1, 0. ],
        [0. , 0. , 0. , 0.3, 0.6, 0.1],
        [0. , 0. , 0. , 0. , 0.3, 0.7]],

       [[0.7, 0.3, 0. , 0. , 0. , 0. ],
        [0.1, 0.6, 0.3, 0. , 0. , 0. ],
        [0. , 0.1, 0.6, 0.3, 0. , 0. ],
        [0. , 0. , 0.1, 0.6, 0.3, 0. ],
        [0. , 0. , 0. , 0.1, 0.6, 0.3],
        [0. , 0. , 0. , 0. , 0.6, 0.4]]])

In [18]:
for run in range(runs):
    temporary_list=[];
    estimated_P_freq = np.zeros((nA,nS,nS));
    for t in range(1,T+1):
        if(t%batch_size==0):
            P_est = np.zeros((nA,nS,nS));
            for s in range(nS):
                if(np.sum(estimated_P_freq[0,s,:])>0):
                    P_est[0,s,:]=estimated_P_freq[0,s,:]/np.sum(estimated_P_freq[0,s,:]);
                if(np.sum(estimated_P_freq[1,s,:])>0):
                    P_est[1,s,:]=estimated_P_freq[1,s,:]/np.sum(estimated_P_freq[1,s,:]);
            print(P_est);
            #input();
            temporary_list.append(P_est);

        action = np.argmax(np.random.multinomial(1,behaviour_policy[state,:]));
        next_state = np.argmax(np.random.multinomial(1,P[action,state,:]));
        estimated_P_freq[action,state,next_state]+=1;
        state = next_state

[[[0.8        0.2        0.         0.         0.         0.        ]
  [0.33333333 0.66666667 0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         1.         0.        ]
  [0.         0.         0.         0.14285714 0.71428571 0.14285714]
  [0.         0.         0.         0.         0.375      0.625     ]]

 [[0.8        0.2        0.         0.         0.         0.        ]
  [0.         0.         1.         0.         0.         0.        ]
  [0.         0.         0.         1.         0.         0.        ]
  [0.         0.         0.         0.5        0.5        0.        ]
  [0.         0.         0.         0.         0.66666667 0.33333333]
  [0.         0.         0.         0.         0.5        0.5       ]]]
[[[0.85714286 0.14285714 0.         0.         0.         0.        ]
  [0.25       0.75       0.         0.         0.         0.        ]
  [0.         0.

In [19]:
nPOL=nS+1;
target_policy = np.ones((nPOL,nS),dtype = np.int8)
for i in range(nPOL-1,0,-1):
    target_policy[nPOL-i-1][0:i] = 0;
print(target_policy)
obj = [find_D_mu(P, policy, nS, nA) for policy in target_policy];
policy_wise_value_func = np.zeros((nPOL,T,runs));

[[0 0 0 0 0 0]
 [0 0 0 0 0 1]
 [0 0 0 0 1 1]
 [0 0 0 1 1 1]
 [0 0 1 1 1 1]
 [0 1 1 1 1 1]
 [1 1 1 1 1 1]]


In [21]:
for index,pol in enumerate(target_policy):
    P = np.array([P_est[pol[s],s,:] for s in range(nS)]);
    print(P);
    #print(i[0],i[1]);
    sd = obj[index].state_distribution_simulated(P,pol)
    vf = sum([R[pol[state],state]*sd[state] for state in range(nS)])
    print(sd)
    print(vf)
    print("policy was:====>",pol);
    print("====================================================");

[[0.85714286 0.14285714 0.         0.         0.         0.        ]
 [0.25       0.75       0.         0.         0.         0.        ]
 [0.         0.33333333 0.33333333 0.33333333 0.         0.        ]
 [0.         0.         0.18181818 0.72727273 0.09090909 0.        ]
 [0.         0.         0.         0.13333333 0.73333333 0.13333333]
 [0.         0.         0.         0.         0.38461538 0.61538462]]
[0.66758242 0.22252747 0.07417582 0.02472527 0.00824176 0.00274725]
0.6675824175824215
policy was:====> [0 0 0 0 0 0]
[[0.85714286 0.14285714 0.         0.         0.         0.        ]
 [0.25       0.75       0.         0.         0.         0.        ]
 [0.         0.33333333 0.33333333 0.33333333 0.         0.        ]
 [0.         0.         0.18181818 0.72727273 0.09090909 0.        ]
 [0.         0.         0.         0.13333333 0.73333333 0.13333333]
 [0.         0.         0.         0.         0.42857143 0.57142857]]
[0.66850069 0.22283356 0.07427785 0.02475928 0.00825